In [8]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
from enrollment_prompt_library import EntityExtraction

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

brt = boto3.client(service_name='bedrock-runtime')

##Processing data to fetch role and content
def data_preprocessing(data):
    convo = ""
    content = json.loads(data["body"])["body"]["transcript"][0]["Content"]
    role = json.loads(data["body"])["body"]["transcript"][0]["ParticipantRole"]
    convo = convo + role + " : " + content
    
    return convo

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    end_char_indices = [i.start() for i in re.finditer("}",data)]
    end_index = end_char_indices[len(end_char_indices)-1]
    result = data[start_index:end_index+1]
    
    return result
    
#Defining function to connect to Bedrock LLM
def load_claude2(bedrock_runtime , prompt , temp , top_p,top_k):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "top_k":top_k,
            "max_tokens_to_sample": 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-v2", body=json.dumps(body), accept="application/json", contentType="application/json"
                 )
        
        response_body = json.loads(response["body"].read())
        completion = response_body.get("completion")

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise

# Lambda handler to intgerate with AWS
def lambda_handler(event,context):
    final_transcript = ""
    for i in range(len(event['Records'])):
        final_transcript += "\n" + data_preprocessing(event['Records'][i])
    prompt_enrollment = EntityExtraction.enrollment_prompt_generator(final_transcript,entities)
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    json_data = data_postprocessing(enrollment_data)
    enrollment_json_object = json.loads(json_data)
    return {"statusCode": 200,"body": json.dumps(enrollment_json_object)}

In [9]:
data = data = {
  "Records": [
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"CUSTOMER\",\"ParticipantRole\":\"CUSTOMER\",\"Content\":\"Sure,my insurance is with Anthem INC.\",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    },
    {
      "messageId": "355c3548-1f03-4c5e-b241-3c6e4bed5601",
      "receiptHandle": "AQEBtGG/BXpQS9fewvrud7Ct+0Pm1fWqFdXUutYN2NKU5Ohukwi/rZLaaKD7mNkMv619WOI1ao9LwvhDO573DROL+wUCQQz1ejI+BaTYWKs629P7sBa8/ND4Bv3SfKPe4p+rMALYuANfO/XoeN6lfjvkNGcSWiyO5PrVyyEPPIKSIg/d/p2viccwRB3s7Q/0TwIxK3iYkSkHoUAOYdcCEQQU1Ia6srdP0D4RCtFXPh4S21F4IljNVF5DYiEFYGX78bPdTVluBqZm9xpjEYw+fOl8YrsIM0p5S9vCBQhouj5rC8Xo8dbrrUo6SAwJqmxWSckmHMda14eyA6zyomtz2xzmxCwdzUVvRMa4J9eMPB+u941W10aK7CP/PvqziaO28aDNuStE6KLOF/2VmgfK9eXF5w==",
      "body": "{\"stream\":\"TRANSCRIPT\",\"streamConnectionId\":\"149edb05-7fd0-48b0-a6e9-3e7b49a0b8d7\",\"body\":{\"transactionId\":\"50b1f7fb-b502-42fa-95f2-281ed6006efe\",\"transcript\":[{\"ParticipantId\":\"AGENT\",\"ParticipantRole\":\"AGENT\",\"Content\":\"Thank you,can you share your policy number? \",\"BeginOffsetMillis\":399340,\"EndOffsetMillis\":399745,\"Id\":\"f10ada6b-dfeb-4569-8b2c-f4e83d60efbc\",\"Sentiment\":\"NEUTRAL\",\"IssuesDetected\":[]}]}}",
      "attributes": {
        "ApproximateReceiveCount": "1",
        "AWSTraceHeader": "Root=1-65d48ed1-196e903e02d83fbd5c5a23bb;Parent=07e7ba1a0a2f4dae;Sampled=0;Lineage=8448a41f:0",
        "SentTimestamp": "1708429009330",
        "SenderId": "AROAVSPTL2UI67PVNI4RP:CH_Kinesis_Transcriber",
        "ApproximateFirstReceiveTimestamp": "1708429009331"
      },
      "messageAttributes": {},
      "md5OfBody": "3488de905e211729d102b02f70a21683",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-east-1:383299343633:ch-agent-assist-sqs",
      "awsRegion": "us-east-1"
    } 
  ]
}

context = " "
lambda_handler(data,context)